# Quantum State Tomography

In [1]:
import qiskit
import qiskit_experiments as qe
tomo = qe.tomography

# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis

# Noisy simulator backend
backend = AerSimulator.from_backend(FakeParis())

## State Tomography Experiment

To run a state tomography experiment we initialize the experiment with a circuit to prepare the state to be measured. We can also pass in an `Operator`, or a `Statevector` to describe the preparation circuit.

In [2]:
# Run experiments

# GHZ State preparation circuit
nq = 2
qc_ghz = qiskit.QuantumCircuit(nq)
qc_ghz.h(0)
qc_ghz.s(0)
for i in range(1, nq):
    qc_ghz.cx(0, i)

# QST Experiment
qstexp1 = tomo.StateTomographyExperiment(qc_ghz)
qstdata1 = qstexp1.run(backend)
qstresult1 = qstdata1.analysis_result(-1)

print('FIT RESULT')
for key, val in qstresult1.items():
    print(f'{key}:', val)

FIT RESULT
state: DensityMatrix([[ 0.49283854+0.00000000e+00j, -0.00195313-2.37630208e-02j,
                -0.00406901-5.04557292e-03j, -0.00097656-4.53125000e-01j],
               [-0.00195313+2.37630208e-02j,  0.02148438+4.33680869e-19j,
                 0.00878906-2.92968750e-03j,  0.02132161-6.02213542e-03j],
               [-0.00406901+5.04557292e-03j,  0.00878906+2.92968750e-03j,
                 0.03710938+1.73472348e-18j,  0.01367187-1.00911458e-02j],
               [-0.00097656+4.53125000e-01j,  0.02132161+6.02213542e-03j,
                 0.01367187+1.00911458e-02j,  0.44856771+0.00000000e+00j]],
              dims=(2, 2))
fitter: linear_inversion
fitter_time: 0.0024261474609375
state_eigvals: [0.92572447 0.046586   0.01704463 0.0106449 ]
raw_eigvals: [0.92572447 0.046586   0.01704463 0.0106449 ]
value: 0.9238281249999996
value_label: state_fidelity
success: True


## Tomography Fitters

The default fitters is `linear_inversion`, which reconstructs the state using *dual basis* of the tomography basis. This will typically result in a non-postive reconstructed state. This state is rescaled to be postive-semidfinite (PSD) by computing its eigen-decomposition and rescaling its eigenvalues using the approach from *J Smolin, JM Gambetta, G Smith, Phys. Rev. Lett. 108, 070502 (2012), [open access](https://arxiv.org/abs/arXiv:1106.5458).

There are several other fitters are included (See API documentation for details). For example if `cvxpy` is installed we can use the `cvxpy_gaussian_lstsq` fitter which allows constraining the fit to be PSD without requiring rescaling.

In [7]:
qstexp1.run_analysis(qstdata1, fitter='cvxpy_linear_lstsq')
qstresult2 = qstdata1.analysis_result(-1)

print('FIT RESULT')
for key, val in qstresult2.items():
    print(f'{key}:', val)

FIT RESULT
state: DensityMatrix([[ 0.49283837+0.00000000e+00j, -0.001953  -2.37630069e-02j,
                -0.00406892-5.04570013e-03j, -0.00097657-4.53124212e-01j],
               [-0.001953  +2.37630069e-02j,  0.02148456-4.33680869e-19j,
                 0.00878896-2.92970733e-03j,  0.02132159-6.02200603e-03j],
               [-0.00406892+5.04570013e-03j,  0.00878896+2.92970733e-03j,
                 0.03710949-1.73472348e-18j,  0.01367174-1.00910654e-02j],
               [-0.00097657+4.53124212e-01j,  0.02132159+6.02200603e-03j,
                 0.01367174+1.00910654e-02j,  0.44856757+0.00000000e+00j]],
              dims=(2, 2))
cvxpy_solver: CVXOPT
cvxpy_status: optimal
fitter: cvxpy_linear_lstsq
fitter_time: 0.059506893157958984
state_eigvals: [0.92572353 0.04658598 0.01704486 0.01064563]
raw_eigvals: [0.92572364 0.04658598 0.01704486 0.01064564]
value: 0.9238271848631854
value_label: state_fidelity
success: True


## Parallel Tomography Experiment

We can also use the `qiskit_experiments.ParallelExperiment` class to run subsystem tomography on multiple qubits in parallel.

For example if we want to perform 1-qubit QST on several qubits at once:

In [8]:
from math import pi
num_qubits = 5
gates = [qiskit.circuit.library.RXGate(i * pi / (num_qubits - 1))
         for i in range(num_qubits)]

subexps = [
    tomo.StateTomographyExperiment(gate, qubits=[i])
    for i, gate in enumerate(gates)
]
parexp = qe.composite.ParallelExperiment(subexps)
pardata = parexp.run(backend)
print(pardata.analysis_result(-1))

{'experiment_types': ['StateTomographyExperiment', 'StateTomographyExperiment', 'StateTomographyExperiment', 'StateTomographyExperiment', 'StateTomographyExperiment'], 'experiment_ids': ['fc504aa5-63f7-4913-b98c-bdb6e25b2050', 'db773ced-43bf-47e2-b43a-051340fccede', '6a9aa414-6c35-4d0e-bf3a-cc8edccff18e', '5a471760-905e-4512-982e-5d7dd02651e4', '3536a471-d3da-48f2-9578-20d2d4969ae4'], 'experiment_qubits': [(0,), (1,), (2,), (3,), (4,)], 'success': True}


View component experiment analysis results

In [5]:
for i in range(parexp.num_experiments):
    expdata = pardata.component_experiment_data(i)
    result = expdata.analysis_result(-1)
    
    print(f'\nPARALLEL EXP {i}: FIT RESULT')
    for key, val in result.items():
        print(f'{key}:', val)


PARALLEL EXP 0: FIT RESULT
state: DensityMatrix([[0.984375  +0.j        , 0.02441406-0.01757812j],
               [0.02441406+0.01757812j, 0.015625  +0.j        ]],
              dims=(2,))
fitter: linear_inversion
fitter_time: 0.0002219676971435547
state_eigvals: [0.98530833 0.01469167]
raw_eigvals: [0.98530833 0.01469167]
value: 0.9843749999999999
value_label: state_fidelity
success: True

PARALLEL EXP 1: FIT RESULT
state: DensityMatrix([[0.85253906+0.00000000e+00j, 0.01757812+3.35937500e-01j],
               [0.01757812-3.35937500e-01j, 0.14746094-8.67361738e-19j]],
              dims=(2,))
fitter: linear_inversion
fitter_time: 0.00011014938354492188
state_eigvals: [0.98728512 0.01271488]
raw_eigvals: [0.98728512 0.01271488]
value: 0.9868264460317541
value_label: state_fidelity
success: True

PARALLEL EXP 2: FIT RESULT
state: DensityMatrix([[0.48535156+0.j        , 0.00976562+0.47070312j],
               [0.00976562-0.47070312j, 0.51464844+0.j        ]],
              dims=(2,))
fi